In [103]:
import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example
import random
import pandas as pd
from sklearn.model_selection import train_test_split  
from sklearn.metrics import classification_report  


In [104]:
df = pd.read_csv('plaintes_dataset.csv')

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  246 non-null    object
 1   categorie    246 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [85]:
df.isnull().sum()

description    0
categorie      0
dtype: int64

In [105]:
df["categorie"].value_counts()

categorie
DECHETS         50
AUTRES          49
AGRESSION       48
VOIRIE          48
CORRUPTION      47
DECHETS          3
CORRUPTION       3
VOIRIE           1
AGRESSION        1
Name: count, dtype: int64

In [87]:
## data prep
### SpaCy attend une liste de tuples tel que chaque élément contient le text brut et un dic 
## [
## ( "texte de la plainte", {"cats": {"AGRESSION":1, "DECHETS":0, …}} ),
##  …
## ]
### {"cats" : labels} ou labels est un dic binaire qui indique pour chaque label si c'est la bonne catégorie(1) ou pas(0)

In [106]:
# 1) liste des labels possibles
LABELS = ["AGRESSION","DECHETS","CORRUPTION","VOIRIE","AUTRES"]

TRAIN_DATA= []

for _,row in df.iterrows():  ## parcourir chaque ligne de dataframe
    text = row["description"]
    true_cat = row["categorie"]

    # 2) on crée un dict de 0 pour tous les labels
    cats = {label: 0 for label in LABELS}

    # 3) on met à 1 la catégorie vrai
    cats[true_cat] = 1

    # 4) on ajoute la tuple à la liste de data
    TRAIN_DATA.append((text,{"cats": cats}))



In [107]:
print("Nombre d'exemples :", len(TRAIN_DATA))
TRAIN_DATA[:3]


Nombre d'exemples : 250


[("Un homme m'a volé mon téléphone près du lycée.",
  {'cats': {'AGRESSION': 1,
    'DECHETS': 0,
    'CORRUPTION': 0,
    'VOIRIE': 0,
    'AUTRES': 0}}),
 ("Les ordures s'accumulent dans la rue sans ramassage.",
  {'cats': {'AGRESSION': 0,
    'DECHETS': 1,
    'CORRUPTION': 0,
    'VOIRIE': 0,
    'AUTRES': 0}}),
 ("Un policier m'a demandé un pot-de-vin.",
  {'cats': {'AGRESSION': 0,
    'DECHETS': 0,
    'CORRUPTION': 1,
    'VOIRIE': 0,
    'AUTRES': 0}})]

# Entrainement du modèle spaCy

In [108]:
nlp = spacy.load("fr_core_news_lg", exclude=["lemmatizer"])

In [109]:
textcat = nlp.add_pipe("textcat", last=True)  

In [110]:
for label in LABELS:
    textcat.add_label(label)

In [111]:
optimizer = nlp.begin_training()  
best_loss = float("inf")  
patience = 2  

for epoch in range(10):  
    random.shuffle(TRAIN_DATA)  
    losses = {}  
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))  
    for batch in batches:  
        examples = [Example.from_dict(nlp.make_doc(text), annots) for text, annots in batch]  
        nlp.update(examples, sgd=optimizer, drop=0.4, losses=losses)  
    print(f"Epoch {epoch+1} — loss={losses['textcat']:.3f}")  

    # Early stopping  
    if losses["textcat"] < best_loss:  
        best_loss = losses["textcat"]  
        patience_counter = 0  
    else:  
        patience_counter +=1  
        if patience_counter >= patience:  
            print("Arrêt anticipé à l'epoch", epoch)  
            break  


Epoch 1 — loss=9.816
Epoch 2 — loss=9.435
Epoch 3 — loss=8.114
Epoch 4 — loss=6.162
Epoch 5 — loss=4.736
Epoch 6 — loss=3.048
Epoch 7 — loss=1.822
Epoch 8 — loss=1.489
Epoch 9 — loss=1.085
Epoch 10 — loss=0.798


In [112]:
nlp.to_disk(output_dir)
print(f"Modèle enregistré dans ./ {output_dir}")

Modèle enregistré dans ./ model_spacy_plaintes


In [115]:
#%% Nouveaux tests - Version 2
new_test_cases = [
    # Cas clairs corrigés
    ("Des gravats de construction bloquent l'accès à l'hôpital Ibn Rochd.", "DECHETS"),
    ("Un élu local demande des votes en échange de travaux publics.", "CORRUPTION"),
    
    # Nouveaux cas complexes
    ("Un inspecteur scolaire réclame un paiement pour modifier des notes.", "CORRUPTION"),  # Corruption éducative
    ("Des déchets de chantier mélangés à de la terre gênent la circulation.", "DECHETS"),    # DECHETS vs VOIRIE
    ("Le trottoir est recouvert de bouteilles en plastique écrasées.", "DECHETS"),           # DECHETS vs VOIRIE
    
    # Cas d'abus d'autorité
    ("Un gardien de prison exige de l'argent pour améliorer les conditions de détention.", "CORRUPTION"),
    
    # Cas limites AUTRES
    ("Des branches d'arbre tombées gênent le passage des piétons.", "AUTRES"),              # Ni DECHETS ni VOIRIE
    ("Un panneau 'Stop' a été volé au carrefour principal.", "AUTRES"),                     # Vandalisme non catégorisé
    
    # Test de confiance
    ("La lune est pleine ce soir.", "AUTRES")  # Phrase hors contexte
]

print(" Tests des corrections et cas complexes :")
evaluate_manual(nlp_eval, new_test_cases)

# Affichage détaillé pour un cas critique
print("\n Détail d'un cas DECHETS/VOIRIE :")
ambiguous_text = "Des débris de brique et de ciment encombrent la chaussée."
doc = nlp_eval(ambiguous_text)
print(f"\nTexte: {ambiguous_text}")
for cat, score in sorted(doc.cats.items(), key=lambda x: x[1], reverse=True):
    print(f"- {cat}: {score:.4f}")

 Tests des corrections et cas complexes :
+-------------------------------------------------------+------------------+--------------+-------------+-----------+
| Texte                                                 | Vrai Catégorie   | Prédiction   |   Confiance | Correct   |
+=======================================================+==================+==============+=============+===========+
| Des gravats de construction bloquent l'accès à l'h... | DECHETS          | DECHETS      |        0.61 | ✅        |
+-------------------------------------------------------+------------------+--------------+-------------+-----------+
| Un élu local demande des votes en échange de trava... | CORRUPTION       | CORRUPTION   |        0.99 | ✅        |
+-------------------------------------------------------+------------------+--------------+-------------+-----------+
| Un inspecteur scolaire réclame un paiement pour mo... | CORRUPTION       | CORRUPTION   |        1    | ✅        |
+----------------